In [1]:
using DataFrames, CSV

In [6]:
using ScikitLearn, MLJ, DecisionTree
#using MultivariateStats, InformationMeasures

In [7]:
using Statistics

In [8]:
#path = ".\\Titanic\\"
path = "./Titanic/"
train = CSV.read(path * "train.csv", DataFrame);
test = CSV.read(path * "test.csv", DataFrame);

In [9]:
s=size(train)
println("num variables = $(s[2]), num observations $(s[1])")
println("variables: ", names(train))

num variables = 12, num observations 891
variables: ["PassengerId", "Survived", "Pclass", "Name", "Sex", "Age", "SibSp", "Parch", "Ticket", "Fare", "Cabin", "Embarked"]


## Column types + missing values

In [21]:
for c in eachcol(train)
    println( typeof(c),' ',eltype(c) )
end

Vector{Int64} Int64
Vector{Int64} Int64
Vector{Int64} Int64
Vector{String} String
PooledArrays.PooledVector{String7, UInt32, Vector{UInt32}} String7
SentinelArrays.SentinelVector{Float64, Float64, Missing, Vector{Float64}} Union{Missing, Float64}
Vector{Int64} Int64
Vector{Int64} Int64
Vector{String31} String31
Vector{Float64} Float64
PooledArrays.PooledVector{Union{Missing, String15}, UInt32, Vector{UInt32}} Union{Missing, String15}
PooledArrays.PooledVector{Union{Missing, String1}, UInt32, Vector{UInt32}} Union{Missing, String1}


In [6]:
ColumnTypes = [(n,eltype(c)) for (n,c) in zip(names(train), eachcol(train))]

12-element Vector{Tuple{String, Type}}:
 ("PassengerId", Int64)
 ("Survived", Int64)
 ("Pclass", Int64)
 ("Name", String)
 ("Sex", String)
 ("Age", Union{Missing, Float64})
 ("SibSp", Int64)
 ("Parch", Int64)
 ("Ticket", String)
 ("Fare", Float64)
 ("Cabin", Union{Missing, String})
 ("Embarked", Union{Missing, String})

## Unique values for Categorical variables

In [7]:
println("Sex values :",unique(train[!,:Sex]))

Sex values :["male", "female"]


In [8]:
for (idx,col) in enumerate(eachcol(train))
    #if eltype(col) <: String <: eltype(col)
    if String <: eltype(col)
        unique_values = unique(col)
        println("Column $(names(train)[idx]) unique Names count :",size(unique_values))
        println(first(unique_values,20))
    end
end

Column Name unique Names count :(891,)
["Braund, Mr. Owen Harris", "Cumings, Mrs. John Bradley (Florence Briggs Thayer)", "Heikkinen, Miss. Laina", "Futrelle, Mrs. Jacques Heath (Lily May Peel)", "Allen, Mr. William Henry", "Moran, Mr. James", "McCarthy, Mr. Timothy J", "Palsson, Master. Gosta Leonard", "Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)", "Nasser, Mrs. Nicholas (Adele Achem)", "Sandstrom, Miss. Marguerite Rut", "Bonnell, Miss. Elizabeth", "Saundercock, Mr. William Henry", "Andersson, Mr. Anders Johan", "Vestrom, Miss. Hulda Amanda Adolfina", "Hewlett, Mrs. (Mary D Kingcome) ", "Rice, Master. Eugene", "Williams, Mr. Charles Eugene", "Vander Planke, Mrs. Julius (Emelia Maria Vandemoortele)", "Masselmani, Mrs. Fatima"]
Column Sex unique Names count :(2,)
["male", "female"]
Column Ticket unique Names count :(681,)
["A/5 21171", "PC 17599", "STON/O2. 3101282", "113803", "373450", "330877", "17463", "349909", "347742", "237736", "PP 9549", "113783", "A/5. 2151", "347082", "3

In [9]:
name="Cumings, Mrs. John Bradley (Florence Briggs Thayer)"
re=r"(\w+),\s*(\w+)\.\s*([\s\w]+)\s\(*([\s\w]+)?\)*"
m = match(re,name)

RegexMatch("Cumings, Mrs. John Bradley (Florence Briggs Thayer)", 1="Cumings", 2="Mrs", 3="John Bradley", 4="Florence Briggs Thayer")

In [193]:
function split_name(name::String)
    re=r"(\w+),\s(\w+)\.\s(\w+\s*\w+)?\s*(\((.+)\))?"
    match(re,name)
end

split_name (generic function with 1 method)

In [229]:
get_title_1(name::String) = split_name(name) != nothing ? m[2] : "unknown"

get_title_1 (generic function with 1 method)

In [233]:
get_title(name::String) = match(r".*, ([\w\s]+)\. .*", name)[1]

get_title (generic function with 1 method)

In [234]:
t=get_title("Cumings, Mrs. John Bradley (Florence Briggs Thayer)")
println(t)
t=get_title("Braund, Mr. Owen Harris")
println(t)
t=get_title("Vestrom, Miss. Hulda Amanda Adolfina")
println(t)
t=get_title("Hewlett, Mrs. (Mary D Kingcome)")
println(t)
t=get_title("Rothes, the Countess. of (Lucy Noel Martha Dyer-Edwards)")

Mrs
Mr
Miss
Mrs


"the Countess"

In [13]:
combine(groupby(train, [:Sex]), :Survived => mean)

,Sex,Survived_mean
,String,Float64
1,male,0.188908
2,female,0.742038


In [14]:
combine(groupby(train, [:Pclass]), :Survived => mean)

,Pclass,Survived_mean
,Int64,Float64
1,3,0.242363
2,1,0.62963
3,2,0.472826


In [38]:
gr=groupby(train, [:Pclass])
pclass_mean_age=combine(gr, :Age => (x->mean(skipmissing(x))) => :Mean)

,Pclass,Mean
,Int64,Float64
1,3,25.1406
2,1,38.2334
3,2,29.8776


In [95]:
pc_age = [ pclass_mean_age[pclass_mean_age.Pclass .==pc,:Mean][1] for pc in 1:3]

3-element Vector{Float64}:
 38.233440860215055
 29.87763005780347
 25.14061971830986

In [83]:
combine(gr, :Age => (x->count(ismissing,x)))

,Pclass,Age_function
,Int64,Int64
1,3,136
2,1,30
3,2,11


In [119]:
t1=copy(train);

In [120]:
size(filter(:Age => ismissing, t1, view=true))

(177, 12)

In [124]:
#filter(row->ismissing(row.Age) && row.Pclass==1, t1, view=true)
size(filter([:Age,:Pclass] => (x,y)-> ismissing(x) && y==1, t1, view=true))


(0, 12)

In [132]:
size(filter([:Age,:Pclass] => (x,y)-> ismissing(x) && y==2, t1, view=true))

(0, 12)

In [136]:
v3 = filter([:Age,:Pclass] => (x,y)->ismissing(x) && y==3, t1, view=true)
size(v3)

(136, 12)

In [178]:
v3[:,:Age] .= pc_age[3]
size(filter([:Age,:Pclass] => (x,y)->ismissing(x) && y==3, t1, view=true))

(0, 12)

In [139]:
filter(row-> ismissing(row.Age) && row.Pclass==1, t1, view=true);

In [140]:
#filter(row-> ismissing(row.Age) && row.Pclass==2, t1, view=true)[:,:Age] .= pc_age[2]

In [168]:
t2=copy(train);
for (key,subg) in pairs( groupby(train, [:Pclass]))
    pclass_mean_age = mean(skipmissing(subg[!,:Age]))
    subg[ismissing.(subg.Age), :Age] .= pclass_mean_age
end

In [171]:
t2=copy(train);
for subg in groupby(train, [:Pclass])
    pclass_mean_age = mean(skipmissing(subg[!,:Age]))
    subg[ismissing.(subg.Age), :Age] .= pclass_mean_age
end

In [172]:
size(t2[(t2.Pclass.==1) .& ismissing.(t2.Age),:Age])

(0,)

In [173]:
size(filter(:Age=>ismissing,t2))

(0, 12)

In [176]:
cov(t2[!,:Age], t2[!,:Survived])

-0.32464428716437516

In [179]:
cor(t2[!,:Age], t2[!,:Survived])

-0.050503588877348855

In [181]:
cov(convert(Array,t2[!,[:Age,:Survived]]))

2×2 Matrix{Float64}:
 174.518     -0.324644
  -0.324644   0.236772

In [182]:
cor(convert(Array,t2[!,[:Age,:Survived]]))

2×2 Matrix{Float64}:
  1.0        -0.0505036
 -0.0505036   1.0

In [184]:
cov(t2[!,:Pclass], t2[!,:Survived])

-0.13770287141073745

In [192]:
split_name.(t2[!,:Name])[2]

RegexMatch("Cumings, Mrs. John Bradley (Florence Briggs Thayer)", 1="Cumings", 2="Mrs", 3="John Bradley", 4="(Florence Briggs Thayer)", 5="Florence Briggs Thayer")

In [235]:
t2.Title = get_title.(t2.Name);

String[]

In [241]:
combine(groupby(t2,[:Title]),nrow)

,Title,nrow
,SubStri…,Int64
1,Mr,517
2,Mrs,125
3,Miss,182
4,Master,40
5,Don,1
6,Rev,6
7,Dr,7
8,Mme,1
9,Ms,1


In [242]:
combine(groupby(t2,[:Title]),:Survived => nrow)

LoadError: MethodError: no method matching nrow(::SubArray{Int64, 1, Vector{Int64}, Tuple{Vector{Int64}}, false})
[0mClosest candidates are:
[0m  nrow([91m::DataFrame[39m) at /home/barabasz/.julia/packages/DataFrames/3mEXm/src/dataframe/dataframe.jl:351
[0m  nrow([91m::SubDataFrame[39m) at /home/barabasz/.julia/packages/DataFrames/3mEXm/src/subdataframe/subdataframe.jl:157